$$
\def\CC{\bf C}
\def\QQ{\bf Q}
\def\RR{\bf R}
\def\ZZ{\bf Z}
\def\NN{\bf N}
$$
# Metabolomics - targeted feature extraction

`FeatureFinderAlgorithmMetaboIdent` performs MS1-based targeted feature
extraction based on user provided compounds, which are specified in an
assay library (a tab-separated text file). Detected `Features` are
stored in a `FeatureMap` which can be stored in a `FeatureXMLFile`. This
tool is useful for the targeted extraction of `Features` for a well
defined set of compounds with known sum formulas and retention times.
For more information on the format of the assay library and available
parameters visit the [FeatureFinderMetaboIdent
documentation](https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/experimental/feature/proteomic_lfq/html/a15547.html).

The pyOpenMS `FeatureFinderAlgorithmMetaboIdent` needs a list of
`FeatureFinderMetaboIdentCompound` objects as an assay libray for it's
`run` function. We could create that list ourselves or use the following
function to read an assay library as `.tsv` file:

<table>
<caption>Coupounds tsv file</caption>
<colgroup>
<col style="width: 32%" />
<col style="width: 19%" />
<col style="width: 9%" />
<col style="width: 9%" />
<col style="width: 9%" />
<col style="width: 9%" />
<col style="width: 9%" />
</colgroup>
<thead>
<tr class="header">
<th>CompoundName</th>
<th>SumFormula</th>
<th>Mass</th>
<th>Charge</th>
<th>RetentionTime</th>
<th>RetentionTimeRange</th>
<th>IsoDistribution</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td>2'-O-methylcytidine</td>
<td>C10H15N3O5</td>
<td>0</td>
<td>1</td>
<td>207.6</td>
<td>0</td>
<td>0</td>
</tr>
<tr class="even">
<td>5-formylcytidine</td>
<td>C10O6N3H13</td>
<td>0</td>
<td>1</td>
<td>269.4</td>
<td>0</td>
<td>0</td>
</tr>
<tr class="odd">
<td>5-methyluridine</td>
<td>C10H14N2O6</td>
<td>0</td>
<td>1</td>
<td>291.6</td>
<td>0</td>
<td>0</td>
</tr>
<tr class="even">
<td>adenosine</td>
<td>C10H13N5O4</td>
<td>0</td>
<td>1</td>
<td>220.8</td>
<td>0</td>
<td>0</td>
</tr>
<tr class="odd">
<td>deoxyadenosine</td>
<td>C10H13N5O3</td>
<td>0</td>
<td>1</td>
<td>243.0</td>
<td>0</td>
<td>0</td>
</tr>
<tr class="even">
<td>inosine</td>
<td>C10H12N4O5</td>
<td>0</td>
<td>1</td>
<td>264.0</td>
<td>0</td>
<td>0</td>
</tr>
</tbody>
</table>

In [ ]:
import csv
# read tsv file and create list of FeatureFinderMetaboIdentCompound
def metaboTableFromFile(path_to_library_file):
    metaboTable = []
    with open(path_to_library_file, 'r') as tsv_file:
        tsv_reader = csv.reader(tsv_file, delimiter="\t")
        next(tsv_reader) # skip header
        for row in tsv_reader:
            metaboTable.append(FeatureFinderMetaboIdentCompound(
                row[0], # name
                row[1], # sum formula
                float(row[2]), # mass
                [int(charge) for charge in row[3].split(',')], # charges
                [float(rt) for rt in row[4].split(',')], # RTs
                [float(rt_range) for rt_range in row[5].split(',')], # RT ranges
                [float(iso_distrib) for iso_distrib in row[6].split(',')] # isotope distributions
            ))
    return metaboTable

Now we can use the following code to detect features with
`FeatureFinderAlgorithmMetaboIdent` and store them in a `FeatureXMLFile`
:

In [ ]:
from urllib.request import urlretrieve

gh = "https://raw.githubusercontent.com/OpenMS/OpenMS/develop"
urlretrieve (gh +"/src/tests/topp/FeatureFinderMetaboIdent_1_input.mzML", "ms_data.mzML")
urlretrieve (gh +"/src/tests/topp/FeatureFinderMetaboIdent_1_input.tsv", "library.tsv")

from pyopenms import *

# load ms data from mzML file into MSExperiment
spectra = MSExperiment()
MzMLFile().load('ms_data.mzML', spectra)

# create FeatureFinderAlgorithmMetaboIdent and assign ms data
ff = FeatureFinderAlgorithmMetaboIdent()
ff.setMSData(spectra)

# read library generate a metabo table with compounds
metabo_table = metaboTableFromFile('library.tsv')

# FeatureMap to store results
fm = FeatureMap()

# edit some parameters
params = ff.getParameters()
params[b'extract:mz_window'] = 5.0 # 5 ppm
params[b'extract:rt_window'] = 20.0 # 20 seconds
params[b'detect:peak_width'] = 3.0 # 3 seconds
ff.setParameters(params)

# run the FeatureFinderMetaboIdent with the metabo_table and store results in fm
ff.run(metabo_table, fm)

# save FeatureMap to file
FeatureXMLFile().store('detected_features.featureXML', fm)

Note: the output file that we have written (`output.featureXML`) is an
OpenMS-internal XML format for storing features. You can learn more
about file formats in the [Reading MS data
formats](other_file_handling.html) section.

We can get a quick overview on the detected features by plotting them
using the following function:

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

def plotDetectedFeatures3D(path_to_featureXML):
    fm = FeatureMap()
    fh = FeatureXMLFile()
    fh.load(path_to_featureXML, fm)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    for feature in fm:
        color = next(ax._get_lines.prop_cycler)['color']
        # chromatogram data is stored in the subordinates of the feature
        for i, sub in enumerate(feature.getSubordinates()):
            retention_times = [x[0] for x in sub.getConvexHulls()[0].getHullPoints()]
            intensities = [int(y[1]) for y in sub.getConvexHulls()[0].getHullPoints()]
            mz = sub.getMetaValue('MZ')
            ax.plot(retention_times, intensities, zs = mz, zdir = 'x', color = color)
            if i == 0:
                ax.text(mz,retention_times[0], max(intensities)*1.02, feature.getMetaValue('label'), color = color)

    ax.set_ylabel('time (s)')
    ax.set_xlabel('m/z')
    ax.set_zlabel('intensity (cps)')
    plt.show()

![image](img/ffmid_graph.png)

[![Launch Binder](./img/launch_binder.jpg)](https://mybinder.org/v2/gh/OpenMS/pyopenms-extra/master+ipynb?urlpath=lab/tree/docs/source/metabolomics_targeted_feature_extraction.ipynb)